# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [8]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [9]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [10]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
[['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88'], ['Jimi Hendrix', 'Logged In', 'Mohammad', 'M', '1', 'Rodriguez', '239.82975', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Woodstock Inprovisation', '200', '1.54328E+12', '88'], ['Building 429', 'Logged In', 'Mohammad', 'M', '2', 'Rodriguez', '300.61669', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Majesty (LP Version)', '200', '1.54328E+12', '88'], ["The B-52's", 'Logged In', 'Gianna', 'F', '0', 'Jones', '321.54077', 'free', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54087E+12', '107', 'Love Shack', '200', '1.54328E+12', '38'], ['Die Mooskirc

In [11]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [12]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [13]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [14]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [15]:
# query1 
query="CREATE TABLE IF NOT EXISTS songs_by_session_and_itemsinsession"
query=query +"""(sessionId int,
itemInSession int,
length float,
song varchar,
artist varchar,
PRIMARY KEY((sessionId),itemInSession));"""
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO  songs_by_session_and_itemsinsession(sessionId,itemInSession,length,song,artist)"
        query = query + "values(%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]),int(line[3]),float(line[5]), line[9],line[0]))

#### Do a SELECT to verify that the data have been inserted into each table

In [17]:
query="""SELECT artist,song,length FROM songs_by_session_and_itemsinsession  WHERE sessionId=338 AND itemInSession=4"""
try:
    data= session.execute(query)
except Exception as e:
    print(e)
for info in data.current_rows:
    print(info.artist,info.song,info.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


the previous cell provides the following infromation:
    1-artist
    2-song
    3-length of the song
from the data provided 
the following were priortized as partition keys :
1-session id (provied every time a song has been played by the user)
2-item in session(number of items 

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [19]:
query="CREATE TABLE IF NOT EXISTS songs_by_user_and_session "
query=query+"""(userId int,
sessionId int,
itemInSession int,
artist varchar,
song varchar,
first_name varchar,
last_name varchar,
PRIMARY KEY((userId,sessionId),itemInSession));"""
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [20]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO  songs_by_user_and_session(userId,sessionId,itemInSession,artist,song,first_name,last_name)"
        query = query + "values(%s,%s,%s,%s,%s,%s,%s);"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))

In [21]:
query="""SELECT artist,song,first_name,last_name FROM songs_by_user_and_session 
WHERE userId=10 AND sessionId=182
ORDER BY itemInSession;"""
try:
    data=session.execute(query)
except Exception as e:
    print(e)
for info in data:
    print(info.artist,info.song,info.first_name,info.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


The previous cell provide the following information:
1-artist
2-song
3-first name of the listener
4-last name of the listener
from the data provided 
the following was priortized as partition keys :
1-user id (unique to every user )
2-session id (provides every time a song has been played by the user)

In [22]:
query="CREATE TABLE IF NOT EXISTS userdata_by_songname"
query=query +""" (song varchar,
userId int,
first_name varchar,
last_name varchar ,
PRIMARY KEY(song,userId));"""
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [23]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO  userdata_by_songname(song,userId,first_name,last_name)"
        query = query + "values(%s,%s,%s,%s);"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query,(line[9],int(line[10]),line[1],line[4]))

In [24]:
query="""SELECT first_name,last_name FROM userdata_by_songname
WHERE song='All Hands Against His Own'"""
try:
    data= session.execute(query)
except Exception as e:
    print(e)
for info in data:
    print(info.first_name,info.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


The previous cell provide the following information:
1-the first name of user
2-the last name of user
from the data provided 
the following was pritiorized as partition key to provide the needed information from the queries:
1-song name (needed to filter the users that listened to the specific song mentioned )
2-user id (unique to every user so it helps in filtering)

### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

In [55]:
session.execute("DROP TABLE IF EXISTS songs_by_session_and_itemsinsession ")
session.execute("DROP TABLE IF EXISTS songs_by_user_and_session ")
session.execute("DROP TABLE IF EXISTS userdata_by_songname ")


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()